In [1]:
import numpy as np

from sudoku_stuff import *

# Resolviendo Sudokus con Simulated Annealing
Inteligencia Artificial - Facundo A. Lucianna - CEIA - FIUBA

En la notebook anterior, intentamos resolver sudokus utilizando el algoritmo de gradiente descendente. Observamos que, aunque el algoritmo encontraba la solución, era necesario realizar muchas iteraciones para obtener buenos resultados.

Ahora veamos si podemos resolverlo usando **Simulated Annealing**. La idea es que el algoritmo, en general, se dirija hacia un estado de menor energía, pero ocasionalmente permita movimientos aleatorios en la dirección contraria. De esta manera, busca escapar de los mínimos locales. La clave es "sacudir" el sistema para salir de un mínimo local sin alejarse demasiado, de modo que podamos llegar al mínimo global.

Resolvamos este problema:

<div>
<img src="./sudoku_7.png" width="300"/>
</div>

El diccionario de **celdas fijas** quedaría de la siguiente forma:

In [2]:
fixed_squares = {
    'A1': 3, 'A3': 4, 'A4': 5, 'A5': 6, 'A7': 9,
    'B1': 1, 'B2': 8, 'B3': 5, 'B6': 9, 'B7': 7,
    'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D2': 2, 'D5': 1, 'D8': 4, 'D9': 9,
    'E2': 4, 'E3': 9, 'E5': 5, 
    'F3': 1, "F4": 9, "F5": 8, "F7": 6, "F8": 7,
    'G1': 4, 'G2': 9, 'G5': 3, 'G9': 7, 
    'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H9': 6,
    'I8': 8,
}

Y la solución que tenemos, que vamos a usar para verificar al final de todo es:

In [3]:
solution = {
    'A1': 3, 'A2': 7, 'A3': 4, 'A4': 5, 'A5': 6, 'A6': 1, 'A7': 9, 'A8': 2, 'A9': 8,
    'B1': 1, 'B2': 8, 'B3': 5, 'B4': 4, 'B5': 2, 'B6': 9, 'B7': 7, 'B8': 6, 'B9': 3,
    'C1': 9, 'C2': 6, 'C3': 2, 'C4': 3, 'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D1': 8, 'D2': 2, 'D3': 7, 'D4': 6, 'D5': 1, 'D6': 3, 'D7': 5, 'D8': 4, 'D9': 9,
    'E1': 6, 'E2': 4, 'E3': 9, 'E4': 2, 'E5': 5, 'E6': 7, 'E7': 8, 'E8': 3, 'E9': 1,
    'F1': 5, 'F2': 3, 'F3': 1, 'F4': 9, 'F5': 8, 'F6': 4, 'F7': 6, 'F8': 7, 'F9': 2,
    'G1': 4, 'G2': 9, 'G3': 6, 'G4': 8, 'G5': 3, 'G6': 2, 'G7': 1, 'G8': 5, 'G9': 7,
    'H1': 2, 'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H7': 3, 'H8': 9, 'H9': 6,
    'I1': 7, 'I2': 5, 'I3': 3, 'I4': 1, 'I5': 9, 'I6': 6, 'I7': 2, 'I8': 8, 'I9': 4,
}

In [4]:
print_state(solution)

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


## Implementado Simulated Annealing

Este algoritmo comienza con una temperatura alta y, a medida que avanza, va enfriándose, de manera similar a cómo se enfría un metal al ser tratado térmicamente.

Aquí tenemos dos nuevos parámetros:
- `initial_temperature`: Es la temperatura inicial con la que arranca el algoritmo. Por defecto, se utiliza el valor 0.01.
- `cooling_rate`: Es la velocidad con la que vamos a enfriar el sistema. La nueva temperatura en cada iteración se actualiza con la fórmula `temperature = temperature*cooling_rate`. Por defecto usamos 0.1.

Luego, tenemos un punto importante: seleccionamos al azar uno de los posibles vecinos. Si la diferencia en el costo entre el estado actual y el vecino es negativa, se acepta el cambio, tal como hacíamos con el gradiente descendente estocástico. Si la diferencia es positiva, se genera un valor aleatorio entre 0 y 1, y si este valor es menor que `exp(-delta_cost / temperature)`, se acepta el cambio. Esta fórmula está basada en la distribución de Boltzmann, que nos dice que, cuanto más grande sea `delta_cost` o más pequeña `temperature`, más difícil será que el cambio sea aceptado.

Veamos la implementación, leyendo cada comentario con atención:

In [5]:
def simulated_annealing_sudoku(initial_state: dict, fixed_squares: dict, max_iterations: int = 1000,
                               initial_temperature: float = 0.01, cooling_rate: float = 0.1):
    """
    Realiza la optimización del Sudoku utilizando Simulated Annealing.

    Args:
        initial_state (dict): El estado inicial del Sudoku.
        fixed_squares (dict): Diccionario que contiene las casillas fijas del Sudoku.
        max_iterations (int, optional): El número máximo de iteraciones permitidas.
                                        Por defecto es 100.
        initial_temperature (float, optional): La temperatura inicial para el algoritmo de Simulated Annealing.
        cooling_rate (float, optional): La tasa de enfriamiento para el algoritmo de Simulated Annealing.

    Returns:
        dict: El mejor estado encontrado después de la optimización.
        float: El costo del mejor estado encontrado
    """
    current_state = initial_state
    best_state = initial_state
    temperature = initial_temperature
    best_cost = cost_function(best_state)

    # Iteramos hasta max_iterations
    for iteration in range(max_iterations):

        # Calculamos la función de costo para el estado actual
        current_cost = cost_function(current_state)

        # Obtenemos los vecinos de un estado
        neib_states = return_neib_states(current_state, fixed_squares)
        amount_neib = len(neib_states)

        # Mientras un estado tengas vecinos
        while(amount_neib > 0):
            # Obtenemos un estado vecino aleatorio
            neib_state = random.choice(neib_states)
            neib_states.remove(neib_state)
            amount_neib -= 1
            
            # Calculamos la función de costo para el estado vecino
            neib_cost = cost_function(neib_state)
    
            # Calculamos el delta de costo entre el estado actual y el vecino
            delta_cost = neib_cost - current_cost
    
            # Si el vecino es mejor o se acepta según la probabilidad de Boltzmann, actualizamos el estado actual
            if delta_cost < 0:
                current_state = neib_state
                break
            else:
                if temperature > 0:
                    if random.random() < math.exp(-delta_cost / temperature):
                        current_state = neib_state
                        break

        # Si termino el anterior ciclo y se visitaron a todos los vecinos y no se acepto ningún cambio
        # se termina el proceso.
        if amount_neib < 1:
            return best_state, best_cost

        # Si el nuevo estado es mejor que el mejor estado encontrado hasta ahora, actualizamos el mejor estado
        if current_cost < best_cost:
            best_state = current_state
            best_cost = cost_function(best_state)

        # Si el costo es cero, significa que estamos en el mínimo. Esto tiene sentido para el caso de Sudoku y la
        # función de costo que implementamos.
        if best_cost == 0:
            break

        # Enfriamos el problema
        temperature *= cooling_rate
        
    # Si terminamos las iteraciones, retornamos el mejor resultado encontrado
    return best_state, best_cost

Ahora, veamos si podemos encontrar la solución en una única ejecución. Para ello, vamos a llamar a una función llamada `execute_search()`, a la cual le pasamos la implementación del algoritmo. Esta función inicializa el sudoku en un estado al azar, aplica la búsqueda y verifica si se ha alcanzado una solución o no.

In [6]:
from processing import execute_search

In [7]:
solution_bool, last_state, initial_state, _ = execute_search(0, simulated_annealing_sudoku, fixed_squares)

In [8]:
print(f"Valor de costo inicial {cost_function(initial_state)} y final {cost_function(last_state)}")
print("Primer estado:")
print_state(initial_state)

print("Último estado encontrado:")
print_state(last_state)

Valor de costo inicial 6.5 y final 0.2
Primer estado:
*---------+---------+---------*
| 3  1  4 | 5  6  8 | 9  5  6 |
| 1  8  5 | 8  1  9 | 7  9  1 |
| 5  6  8 | 2  7  8 | 4  1  5 |
*---------+---------+---------*
| 7  2  8 | 4  1  7 | 6  4  9 |
| 3  4  9 | 9  5  2 | 5  3  2 |
| 8  7  1 | 9  8  5 | 6  7  9 |
*---------+---------+---------*
| 4  9  2 | 6  3  3 | 5  3  7 |
| 2  1  8 | 7  4  5 | 9  7  6 |
| 9  2  7 | 5  3  1 | 7  8  9 |
*---------+---------+---------*
Último estado encontrado:
*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 6  2  7 | 2  1  3 | 5  4  9 |
| 8  4  9 | 6  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


In [9]:
print("El estado encontrado es solución?")
if solution_bool:
    print("El estado que encontramos verifica que realmente es la solución")
else:
    print("El estado que encontramos no es solución")

El estado encontrado es solución?
El estado que encontramos no es solución


Vemos que el algoritmo no está encontrando la **solución**. Ahora, podemos probar repetir 500 ejecuciones de la busqueda, comenzando desde diferentes puntos iniciales, para ver si eventualmente llegamos a la solución. Para acelerar el proceso, aprovecharemos que tenemos **CPUs multinúcleo**.

Para ello, vamos a llamar a la función `parallel_sudoku_search()`, a la cual le pasaremos la función de búsqueda y el número de iteraciones que queremos realizar.

In [10]:
from processing import parallel_sudoku_search

# Debemos llamar a la función de busqueda desde un archivo .py sino los threads no pueden recibir a la función desde la notebook directamente.
from search_methods import simulated_annealing_sudoku

In [11]:
results = parallel_sudoku_search(simulated_annealing_sudoku, fixed_squares, max_iterations=500)

  0%|          | 0/500 [00:00<?, ?it/s]

Veamos si algún proceso encontró la solución:

In [12]:
show_solution = True
for res in results:
    # Acá nos devuelve el booleano de si encontró la solución o no
    is_solution = res[0]
    # Este es el últimos estado encontrado en esta iteración
    last_state = res[1]
    # Este es el estado desde donde partio
    initial_state = res[2]
    # Este es el identificador de cual iteración se obtuvo la solución
    process_id = res[-1]

    if is_solution:
        if show_solution:
            print_state(last_state)
            show_solution = False
        print(f"En la iteración {process_id} se encontró la solución.")

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*
En la iteración 20 se encontró la solución.
En la iteración 27 se encontró la solución.
En la iteración 37 se encontró la solución.
En la iteración 40 se encontró la solución.
En la iteración 46 se encontró la solución.
En la iteración 57 se encontró la solución.
En la iteración 69 se encontró la solución.
En la iteración 101 se encontró la solución.
En la iteración 117 se encontró la solución.
En la iteración 131 se encontró la solución.
En la iteración 151 se encontró la solución.
En la iteración 153 se encontró la solución.
En la iteración 232 se encontró la solución.
En la 

**Nota**: Si usamos una temperatura inicial igual a cero, obtenemos una implementación equivalente al gradiente descendente estocástico.

Con este método, es más fácil encontrar la solución. Mientras que con gradiente descendente tuvimos que repetir el experimento muchas veces hasta dar con la solución, con **Simulated Annealing** generalmente basta con unas pocas ejecuciones para llegar a la solución. La temperatura permite escapar de algunos mínimos locales que existen en la función de costo. Lo negativo de este método es que lleva tiempo encontrar los valores adecuados de temperatura y tasa de enfriamiento.